In [1]:
from tensorflow import keras
from tensorflow.keras.layers import *
from prepare_data import *
from datetime import datetime

In [1]:
import tensorflow as tf
import numpy as np
import albumentations as A

print(tf.__version__)
print(np.__version__)
print(A.__version__)

2.8.0
1.22.3
1.1.0


In [2]:
dataset = Dataset.eighty
dataset_name = str(dataset).split(".")[1]
current_time = datetime.now().strftime("%d%m%Y-%H%M%S")

In [3]:
raw_train, raw_val = prep_dataset(dataset, 8)
train_gen, val_gen = prep_multi_img_ensemble_dataset(dataset, raw_train, raw_val, 8, 3)

Found 27831 files belonging to 80 classes.
Using 23657 files for training.
Found 27831 files belonging to 80 classes.
Using 4174 files for validation.


In [4]:
extractor_path = f"../model-saves/extractors/{dataset_name}/{dataset_name.upper()}-EXTRACTOR/extractor/savefile.hdf5"
extractor1 = keras.models.load_model(extractor_path).layers[0].layers[-1]
extractor1.trainable = True
for layer in extractor1.layers:
    layer._name += "_1"

extractor2 = keras.models.load_model(extractor_path).layers[0].layers[-1]
extractor2.trainable = False
for layer in extractor2.layers:
    layer._name += "_2"

extractor3 = keras.models.load_model(extractor_path).layers[0].layers[-1]
extractor3.trainable = False
for layer in extractor3.layers:
    layer._name += "_3"

In [5]:
concat_layer = keras.layers.concatenate([extractor1.output, extractor2.output, extractor3.output])
dropout_layer = Dropout(0.5)(concat_layer)
dense_layer = Dense(1000, activation='relu')(dropout_layer)
predictions = keras.layers.Dense(train_gen.num_classes(), activation='softmax')(dense_layer)

In [6]:
classifier_model = keras.Model(inputs=[extractor1.input, extractor2.input, extractor3.input], outputs=predictions)
classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
logdir = "../logs/unfiltered/3_img_ensemble_{0}_{1}/classifier".format(str(dataset), current_time)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model_path = "../model-saves/unfiltered/3_img_ensemble_{0}_{1}/classifier/savefile.hdf5".format(str(dataset), current_time)
model_save_callback = keras.callbacks.ModelCheckpoint(filepath=model_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

In [8]:
class ShuffleCallback(keras.callbacks.Callback):
    def __init__(self, generator):
        self._generator = generator
        
    def on_epoch_end(self, epoch, logs=None):
        self._generator.shuffle()
    
train_shuffle_callback = ShuffleCallback(train_gen)
val_shuffle_callback = ShuffleCallback(val_gen)

In [9]:
classifier_model.fit(train_gen, validation_data=val_gen, callbacks=[tensorboard_callback, model_save_callback, train_shuffle_callback, val_shuffle_callback], epochs=20)

Epoch 1/20
983/983 [==============================] - ETA: 0s - loss: 2.2244 - accuracy: 0.5193
Epoch 1: val_accuracy improved from -inf to 0.86755, saving model to ../model-saves/unfiltered/3_img_ensemble_Dataset.eighty_15052022-200138/classifier\savefile.hdf5
983/983 [==============================] - 254s 240ms/step - loss: 2.2244 - accuracy: 0.5193 - val_loss: 0.4895 - val_accuracy: 0.8675
Epoch 2/20
983/983 [==============================] - ETA: 0s - loss: 0.5963 - accuracy: 0.8360
Epoch 2: val_accuracy improved from 0.86755 to 0.91023, saving model to ../model-saves/unfiltered/3_img_ensemble_Dataset.eighty_15052022-200138/classifier\savefile.hdf5
983/983 [==============================] - 222s 225ms/step - loss: 0.5963 - accuracy: 0.8360 - val_loss: 0.2438 - val_accuracy: 0.9102
Epoch 3/20
983/983 [==============================] - ETA: 0s - loss: 0.3468 - accuracy: 0.8976
Epoch 3: val_accuracy improved from 0.91023 to 0.93157, saving model to ../model-saves/unfiltered/3_img_ens